In [ ]:
"""Hartree-Fock Procedure for Approximate Quantum Chemistry"""

__authors__ = ["Ashley Ringer McDonald","Dominic A. Sirianni", "Tricia D. Shepherd", "Sean Garrett-Roe"]
__email__   = ["armcdona@calpoly.edu", "sirianni.dom@gmail.com", "profshep@icloud.com", "sgr@pitt.edu"]
__credits__ = ["Daniel G.A. Smith"]
__copyright__ = "(c) 2008-2020, The Psi4Education Developers"
__license__   = "BSD-3-Clause"
__date__      = "2020-07-28"

In [ ]:
import psi4
import numpy as np
from scipy import linalg as splinalg

Dans ce TP, vous allez construire et diagonaliser la matrice de Fock afin de déterminer les coefficients 
et les énergies des OM d'une molécule. Nous utiliserons les fonctions du logiciel de chimie quantique 
Psi4 pour calculer les intégrales dont nous avons besoin.  
Le carnet de notes suivant vous guidera dans la configuration de votre molécule, 
l'établissement de la base, la génération et la diagonalisation de la matrice de Fock. 
Assurez-vous d'exécuter chaque cellule au fur et à mesure que vous avancez dans le carnet de notes.

# I. La procédure Hartree-Fock
L'équation de Schrödinger a la structure d'une équation aux valeurs propres :

$$\hat{H}|\psi\rangle = E|\psi\rangle$$

Dans la théorie Hartree-Fock, elle est réexprimée en termes de matrice de Fock, $F$, de matrice d'amplitudes de fonction d'onde pour chaque MO, $C$, et de matrice de recouvrement, $S$,

$$FC = SCE.\qquad(\text{2})$$

La matrice de Fock d'un système à couche fermée est la suivante :
$$
F = H + 2J - K
$$

où $H$ est l'hamiltonien du "noyau" à un électron, $J$ est la matrice intégrale de Coulomb et $K$ est la matrice intégrale d'échange. Les définitions de $J$ et $K$ dépendent des coefficients, $C$. Nous voyons ici le principe central de la procédure SCF (Self-Consistent-Field) : pour obtenir la matrice de Fock, nous avons besoin de la matrice des coefficients, mais pour calculer la matrice des coefficients, nous avons besoin de la matrice de Fock. Lorsque $S$ n'est pas égal à la matrice d'identité (c'est-à-dire que la base n'est pas orthonormée), il s'agit alors d'un problème de pseudo-valeur propre qui est encore plus difficile à résoudre. C'est le but de ce TP.

Nous allons :
- revoir la notation de Dirac 
- introduire la matrice de recouvrement 
- apprendre à construire une matrice d'orthogonalisation
- apprendre à calculer la densité
- apprendre à calculer les matrices intégrales de Coulomb et d'échange
- apprendre à diagonaliser la matrice de Fock
- construire une procédure itérative pour faire converger l'énergie HF

# II. Orthogonalisation de la base d'Orbitales Atomiques (OA)

Petit rappel sur la notation de Dirac :

La fonction d'onde, $\psi(x)$ peut être représentée comme un *vecteur colonne*, $|\psi\rangle$ . Le conjugué complexe de la fonction d'onde, $\psi^*(x)$ est également représenté par un vecteur qui est le transposé du conjugué complexe de $|\psi\rangle$. 

\begin{align}
\psi(x)&\rightarrow|\psi\rangle\quad\text{column vector}\\
\psi^*(x)&\rightarrow\langle\psi|\quad\text{row vector}
\end{align}

La normalisation d'une fonction d'onde $\psi(x)$ s'écrit comme une intégrale

$$\int\psi^*(x)\psi(x)\;\mathrm{d}x=1.$$

En notation de Dirac, elle est remplacée par une équation vectorielle

$$\langle\psi|\psi\rangle=1.$$

L'orthogonalité de deux fonctions d'onde, $\psi_i(x)$ et $\psi_j(x)$, qui, en notation intégrale, s'écrit 

$$\int\psi_i^*(x)\psi_j(x)\;\mathrm{d}x=0,$$

devient, en notation de Dirac,

$$\langle\psi_i|\psi_j\rangle=0.$$
 
### Calcul : 
Définissez deux vecteurs, `phi1` et `phi2`, avec deux composantes chacun, qui sont normalisés, dans le sens $\langle\phi_i|\phi_i\rangle=1$, et orthogonaux dans le sens que $\langle\phi_i|\phi_j\rangle=0$. On définit un vecteur `v` avec les deux éléments `1` et `2` par la commande `v=np.array([1,2])`.

In [ ]:
# define a first basis vector and a second, orthogonal vector
phi1 = #<entrer votre formule ici>
phi2 = #<entrer votre formule ici>

print(F'Phi1: {phi1}')
print(F'Phi2: {phi2}')
print()#empty line
print(F'Produit scalaire : {phi1.dot(phi2)}')

Note : Les commandes Numpy pour les opérations vectorielles supposent que `v`= vecteur :

produit interne (produit scalaire) de deux vecteurs : `v.dot(v)`

conjugué complexe d'un vecteur : `v.conj()`

Produit interne de $v^\dagger v$ : `v.conj().dot(v)`


### Calcul : 
Utilisez les commandes ci-dessus pour montrer que `phi1` est normalisé.

In [ ]:
# Normalisation de phi1

phi1_norm = #<entrer votre formule ici>

print(F'<phi1|phi1> = {phi1_norm}')

### Calcul : 
De même, entrez les trois calculs restants ci-dessous pour montrer que `phi1` et `phi2` sont orthonormés.

In [ ]:
# Normalisation de phi2

print(F'<phi2|phi2> = {}')

# Orthogonalité

print(F'<phi1|phi2> = {}')

print(F'<phi2|phi1> = {}')

## III. Intégrales de recouvrement
Pour un ensemble de fonctions de base, $\phi_i(\tau)$, où $\tau$ désigne toutes les coordonnées de toutes les particules, nous pouvons calculer les intégrales de recouvrement entre les fonctions de base de la manière suivante

$$S_{ij}=\int {\rm d}\tau\; \phi_i^*(\tau)\phi_j(\tau).$$

#### Question :  définissez $S_{ij}$ en utilisant la notation de Dirac

**Réponse :** 

#### Question :  pour une base orthonormée, à quoi ressemble la matrice des intégrales de recouvrement, `S` ?

**Réponse :** 

### Calculer les termes `S_ij` en utilisant les vecteurs de base `phi1` et `phi2`. 

In [ ]:
# calculate the overlap (inner product) of the vectors 
S_11 = #<entrer votre formule ici>
S_12 = 
S_21 =  
S_22 = 
print('Les éléments ij de S:')
print(S_11,S_12)
print(S_21,S_22)
print()


## IV. Construction de la matrice de recouvrement
Ces intégrales de recouvrement, $S_{ij}$, peuvent être interprétées comme les éléments de la $i$ème ligne et de la $j$ème colonne d'une matrice, $S$. Soit une matrice, $S$, constituée des intégrales de recouvrement $S_{ij}$. Nous pouvons construire $S$ systématiquement de la manière suivante. D'abord, construisons une matrice, $B$, composée de nos vecteurs de base comme colonnes,

$$ B = \left(\begin{array}{ccc}|& |&|\\ \phi_1 &\phi_2&\phi_3\\|&|&|\end{array}\right).$$

Nous utiliserons le symbole $\dagger$ pour indiquer le conjugué complexe de la transposée d'une matrice. Ainsi,

$$B^\dagger = (B^T)^*.$$

In [ ]:
# constructino de la matrice de recouvrement à partir de la matrice des vecteurs de base
vector_length = phi1.size #dimension de l'espace vectoriel
phi1_column = phi1.reshape(vector_length,1) #cela transforme phi1 en vecteur colonne
phi2_column = phi2.reshape(vector_length,1)

# rassembler (concaténer) les vecteurs dans la matrice B
B = np.concatenate((phi1_column,phi2_column),axis=1)
print(F'La matrice B:\n{B}')

B_dagger = B.conj().T
print(F'La matrice B^\dagger:\n{B_dagger}')

Maintenant, en multipliant les lignes de $B^\dagger$ par les colonnes de $B$ (multiplication matricielle normale), on obtient une matrice des intégrales de recouvrement aux emplacements corrects. Nous avons défini la matrice $S$.

$$B^\dagger B =\left(\begin{array}{ccc}-& \phi_1^*&-\\-& \phi_2^* &-\\-&\phi_3&-\end{array}\right) \left(\begin{array}{ccc}|& |&|\\ \phi_1 &\phi_2&\phi_3\\|&|&|\end{array}\right)\equiv S.$$

Remarque : les commandes Numpy pour les opérations matricielles supposent que `v` = vecteur, et `M` = matrice :

produit vectoriel matriciel : `M.dot(v)`

Produit matriciel de la matrice : `M.dot(M)`

Conjugué complexe de la matrice : `M.conj()`

transposée de la matrice : `M.T`

### Calcul : 
Utilisez `B` et `B_dagger` et les règles matricielles ci-dessus pour calculer la matrice `S`.

In [ ]:
# calcul de S à partir de la matrice des vecteurs de base
S = #<entrer votre formule ici>
print(F'La matrice des vecteurs propres en colonne B =\n {B} \n\nand S = B^\dagger B =\n{S}\n')

## V. Notation implicite de la somme d'Einstein
La multiplication matricielle est définie par

$$(AB)_{pq}= \sum_i A_{p,i}B_{i,q}\qquad\text{sommation implicite}$$$

Notez qu'il y a un indice répété, $i$, dans la sommation. En notation de sommation implicite, notation d'Einstein, nous n'écrivons pas la $\sum$ et traitons la sommation comme comprise. 

$$(AB)_{pq}=A_{p,i}B_{i,q}\qquad\text{sommation implicite}$$

En utilisant la sommation implicite pour le cas présent, $B^\dagger B$ donne

S_{pq}=(B^\dagger B)_{pq}= (B^\dagger)_{p,i}B_{i,q}\qquadtext{sommation implicite}$$.
$$= (B^*)_{i,p}B_{i,q}\qquad\text{sommation implicite}$$$

où $B^*$ est le conjugué complexe de $B$ (sans transposition). Notez que les deux ensembles d'indices, $(i,p)$ et $(i,q)$, dans les matrices d'entrée deviennent un seul ensemble, $(p,q)$, dans le produit. 

*(Petite aparté sur la programmation Python : il existe une fonction pratique dans `numpy` appelée `einsum()`, qui est l'un des joyaux de la bibliothèque numpy.  En bref, `einsum` vous permet d'effectuer diverses combinaisons de multiplications, de sommations et de transpositions de matrices très efficacement.  Un bon tutoriel sur `einsum` peut être trouvé à http://ajcr.net/Basic-guide-to-einsum/.  Pour spécifier les opérations que vous voulez faire, vous utilisez la convention **Ein**stein **Sum**mation.)*

Pour calculer la somme 

$$S_{pq} = (B^*)_{i,p}B_{i,q}\qquad\text{sommation implicite}$$$

utilisez `S=np.einsum('ip,iq->pq',B.conj(),B)`, comme démontré ci-dessous. Nous allons utiliser `einsum()` à plusieurs endroits.

Essayons ceci avec un simple ensemble de base de deux vecteurs (peut-être) orthonormaux.

#### Question : décrivez comment la notation de la commande `np.einsum` coïncide avec la formule de sommation implicite écrite ci-dessus.

****Réponse **** 

### Calcul : 
Utilisez la fonction `np.einsum()` pour calculer la matrice `S`, et confirmez que votre réponse est la même que ci-dessus.

In [ ]:
# calcul de S à partir de la somme d'Einstein 
S = #<entrer votre formule ici>
print(F'S à partir de la notation d'Einstein:\n{S}')

#### Question : Proposez une base orthonormale différente, modifiez `phi1` et `phi2`, et vérifiez que `S` a toujours la même forme. Il y a une infinité de choix possibles. Ce n'est pas complexe... ou *est*-ce que ça l'est ?

#### Question : proposez ce qui arrivera à `S` si les vecteurs ne sont pas orthonormés. 
### Calcul : 
Tester votre prédiction

## VI. Base d'orbitales atomiques Gaussiennes

H$_2$O est une molécule petite mais intéressante à utiliser dans notre exploration. 

#### Question :  Répondez aux questions suivantes pour la molécule H$_2$O :
##### Combien d'électrons y a-t-il au total ?
##### A combien d'orbitales moléculaires occupées vous attendez-vous ?

Avant de commencer à mettre en œuvre la procédure HF, nous devons spécifier la molécule et le jeu de base que nous allons utiliser.  Nous allons également définir l'utilisation de la mémoire pour notre calcul et le nom du fichier de sortie.  

In [ ]:
# ==> Set Basic Psi4 Options <==
# Spécification de la mémoire
psi4.set_memory('500 MB')
numpy_memory = 2 # Aucun tableau NumPy ne peut dépasser 2 MB en taille

# nom du fichier de sortie
psi4.core.set_output_file('output.dat', False)

# on spécifie la base
# basis = 'cc-pvdz'
basis = 'sto-3g'


# On fixe les options du calcul
psi4.set_options({'basis': basis,
                  'scf_type': 'pk',
                  'e_convergence': 1e-8})


# ==> Définition de la molécule <==
# Définir notre modèle de molécule d'eau
# nous allons déformer la molécule plus tard, ce qui peut nécessiter une symétrie C1
mol = psi4.geometry("""
O
H 1 1.1
H 1 1.1 2 104
symmetry c1
""")

# calcul d'énergie

SCF_E_psi = psi4.energy('scf')
psi4.core.clean()

print(f"L'énergie  Hartree-Fock de l'état fondamental de l'eau est : {SCF_E_psi} Eh")

Ensuite, nous devons construire la fonction d'onde à partir des fonctions de base.  Nous stockons la fonction d'onde dans une variable appelée `wfn`. Nous utilisons la fonction `nalpha()` fournie par l'objet fonction d'onde que nous avons créé plus haut, `wfn`, pour déterminer le nombre d'orbitales de spin alpha, qui seront des orbitales doublement occupées pour les systèmes à couche fermée. Nous enregistrons cette réponse comme une variable appelée `ndocc` (nombre d'orbitales doublement occupées).  

### Calcul
Exécutez le code ci-dessous et confirmez que le nombre d'orbitales doublement occupées correspond à votre attente pour la molécule que vous avez choisie.

In [ ]:
# ==> Compute static 1e- and 2e- quantities with Psi4 <==
wfn = psi4.core.Wavefunction.build(mol, psi4.core.get_global_option('basis'))

# nombre d'orbitales de spin alpha (doublement occupées pour les systèmes à couche fermée)
ndocc = wfn.nalpha()
nbf = wfn.basisset().nbf()

print(F'Nombre d'orbitales occupées : {ndocc}')
print(F'Nombre de fonctions de base: {nbf}') 

Ensuite, nous allons examiner le jeu de base des orbitales atomiques. Pour ce faire, nous devons mettre en place une structure de données, appelée classe, pour calculer les intégrales moléculaires. (Nous appellerons cette structure de données `mints` (Molecular INTegralS). Nous utilisons la fonction `ao_overlap` pour calculer les intégrales de recouvrement entre toutes les fonctions de base AO.  Nous mettons le résultat dans un tableau numpy appelé `S`. 

(A propos de la programmation Python : `asarray()` est un cas spécial de `array()` qui ne copie pas les tableaux lorsqu'il est compatible et qui convertit les sous-classes de tableaux en ndarrays de la classe de base. https://stackoverflow.com/questions/14415741/what-is-the-difference-between-numpys-array-and-asarray-functions)_ 

In [ ]:
# Construction d'un objet "intégrales moléculaires".
mints = psi4.core.MintsHelper(wfn.basisset())

# Matrice de recouvrement en tant qu'objet Matrice psi4
S_matrix = mints.ao_overlap()

# Matrice de recouvrement convertie en tableau np
S = np.asarray(S_matrix) 

print(F'La forme de S est {S.shape}')

#### Question : Expliquez la forme (nombre de lignes et de colonnes) de `S` en fonction de la base d'OA que nous avons choisie.

Examinons le contenu de `S`.

In [ ]:
print(S) #la matrice complète peut être quelque peu difficile à lire en fonction de la base

In [ ]:
# Regardons les premiers éléments
def peak(S,nrows=4,ncols=4):
    print(F'Voici un aperçu des premiers éléments {nrows} x {ncols} de la matrice :\n{S[:nrows,:ncols]}')
    
peak(S)

#### Question :  En vous basant sur vos observations de `S` dans la base AO, répondez aux questions suivantes
##### Qu'indiquent les éléments diagonaux de `S` ?
##### Qu'indiquent les éléments hors diagonale de `S` ?
##### Le jeu de base des orbitales atomiques gaussiennes forme-t-il une base orthonormée ? 

Nous pouvons également effectuer ce test à l'aide d'un programme, avec quelques astuces en Python. Construisez un tableau de la même taille que le tableau de recouvrement (`S`) qui a des 1 le long de la diagonale et des 0 partout ailleurs. Comparez ensuite ce tableau au tableau `S` pour déterminer si la base AO est orthonormée.

In [ ]:
# Exemple de test d'orthogonalité

# on définit une fonction
def isBasisOrthonormal(S):
    # on récupère les lignes de S
    size_S = S.shape[0] 
    
    # On construit une matrice identité I 
    identity_matrix = np.eye(size_S) 

    # Tous les éléments de S sont-ils numériquement proches de la matrice d'identité ? 
    # Nous ne testons pas l'égalité car il peut y avoir de très petites différences numériques qui ne nous intéressent pas. 
    # numériques minimes dont nous ne nous soucions pas
    orthonormal_check = np.allclose(S, identity_matrix)

    print(F'Q:(T/F) La base d'OA est-elle orthonormée? A: {orthonormal_check}')
    return orthonormal_check

# use the function
isBasisOrthonormal(S)

#### Question : Le résultat est-il conforme à ce que vous avez déterminé ci-dessus ? Expliquez.

### VII. Une matrice d'orthogonalisation
Rappelons que si nous avions utilisé les fonctions d'onde de l'atome d'hydrogène comme ensemble de base, les fonctions d'onde de l'AO seraient toutes orthonormées. Puisque nous avons utilisé un ensemble de base de fonctions gaussiennes, cela peut ne pas être le cas. Nous allons maintenant présenter quelques outils pour y remédier !

Puisque notre ensemble de base AO n'était pas orthonormé, nous cherchons à construire une matrice d'orthogonalisation, $A$, telle que ${\bf A}^{\dagger}{\bf SA} = {\bf 1}$. 

**Motivation:** Si ${\bf A}$ et ${\bf S}$ étaient des nombres réels $a$ et $s$ (et non des matrices), cette question serait simple à résoudre. Tout d'abord, $a^\dagger=a$ car un nombre réel est identique à sa transposition hermitienne. Par simple algèbre, nous pouvons résoudre a,

$$a^\dagger s a=1$$
$$a^\dagger s a=a s a=a^2s=1$$
$$\Rightarrow{}a=s^{-1/2}$$ 

En algèbre linéaire (avec des matrices au lieu de nombres), c'est plus compliqué, mais numpy et la classe mints peuvent s'occuper des détails pour nous ! En laissant de côté les détails, nous allons calculer

$${\bf A}={\bf S}^{-1/2}$$.


#### Calcul : 
Utilisez la fonction `np.linalg.inv()` pour calculer l'inverse de `S`, et la fonction `splinalg.sqrtm()` pour prendre sa racine carrée (matricielle). Exécutez le code ci-dessous et examinez la matrice `A`.

In [ ]:
# ==> Construire la matrice d'orthogonalisation AO <=>

# inverse de S en utilisant np.linalg.inv
# matrice racine carrée de l'inverse de S en utilisant splinalg.sqrtm

A = #<entrer votre formule ici>

peak(A)

#### Question : Qu'observez-vous à propos des éléments de `A` ? La matrice est-elle réelle ou complexe ? La matrice est-elle symétrique ou non ?

Notre base $B$ n'est pas orthonormée, nous voulons donc prendre des combinaisons linéaires de ses colonnes pour créer une nouvelle base, $B'$, qui est orthonormée. Nous définissons une nouvelle matrice, $A$, en fonction de cette transformation,

$$B' = BA.$$

La nouvelle matrice de recouvrement sera

\begin{align}
 S' &= B'^\dagger B',\\
 &= (BA)^\dagger (BA),\\
&= A^\dagger B^\dagger BA,\\
&= A^\dagger S A.
\end{align}

La matrice $A$ fait la combinaison linéaire propre des colonnes de $B$ et $A^\dagger$ fait les combinaisons linéaires des lignes de $B^\dagger$. Il s'agit d'une structure très courante des matrices de transformation. Puisque $S$ est réelle et symétrique, $A$ est également réelle et symétrique, donc $A^\dagger=A$. La transformation devient simplement

$$S' = A S A.$$

#### Calcul
Utilisez la matrice d'orthogonalisation `A` pour transformer la matrice de chevauchement, `S`. Vérifiez la matrice de recouvrement transformée, `S_p`, pour vous assurer qu'elle représente une basis orthonormale.

In [ ]:
# Transformer S avec A et affecter le résultat à S_p

S_p = #<entrer votre formule ici>

isBasisOrthonormal(S_p)

#### Question : Le produit A S A ne prend pas la transposée conjuguée complexe de A. Quelles sont les conditions (propriétés de A) qui font que cela est correct ?

## VIII. La matrice de Fock transformée en une base orthonormée

Nous pouvons maintenant revenir à l'équation 2 en utilisant notre matrice d'orthogonalisation $A$. Une astuce courante en algèbre linéaire consiste à "insérer un". Dans ce cas, la matrice $A$ multipliée par son inverse est, par définition, la matrice identité, $AA^{-1}={\bf1}$. Nous pouvons placer ce facteur "un" n'importe où dans une équation qui nous est utile, et, ensuite, typiquement nous regroupons les termes de la manière que nous voulons. Dans ce cas, nous en insérons un entre $FC$ et $SC$.


\begin{align}
FC&=SCE\\
F({\bf1})C&=S({\bf1})CE\\
FAA^{-1}C&=SAA^{-1}CE
\end{align}

En multipliant à gauche par $A$, on obtient alors

$$
AFAA^{-1}C=ASAA^{-1}CE
$$

Nous pouvons reconnaître la transformation $S'=ASA$ du côté droit et définir de manière similaire $F'=AFA$ du côté gauche. Enfin, nous définissons une matrice de coefficients transformée, $C'=A^{-1}C$. Notre équation de Fock transformée est la suivante

\begin{align}
F'C'&=S'C'E,\\
&=C'E.
\end{align}

La dernière ligne suit parce que $S'={\bf1}$ dans notre nouvelle base. Nous avons maintenant un problème de valeur propre que nous pouvons résoudre par diagonalisation de la matrice.

Dans l'expression

$$C'=A^{-1}C,$$

la matrice $A^{-1}$ transforme les coefficients, $C$, dans l'ensemble de base orthogonalisé. Nous aurons également besoin d'un moyen de transformer ces coefficients, $C'$, pour les ramener dans la base AO originale.

#### Question : En vous appuyant sur la définition de $C'$, proposez une définition de $C$ en fonction de $A$ et $C'$. Justifiez votre équation.

**Réponse:**

## IX. "Guess" initial de la matrice de Fock est le hamiltonien à un électron.
Pour obtenir la matrice de Fock, nous avons besoin de la matrice des coefficients, mais pour calculer la matrice des coefficients, nous avons besoin de la matrice de Fock.  Nous commençons donc par une estimation de la matrice de Fock, qui est la matrice hamiltonienne centrale.

In [ ]:
# Build core Hamiltonian
T = np.asarray(mints.ao_kinetic())
V = np.asarray(mints.ao_potential())
H = T + V

#### Calcul : 
Dans la cellule ci-dessous, utilisez la matrice Hamiltonienne de base comme estimation initiale pour la matrice de Fock. Transformez-la avec la même matrice A que vous avez utilisée ci-dessus.  Pour calculer les valeurs propres, `vals`, et les vecteurs propres, `vecs`, de la matrice `M` en utilisant `vals, vecs = np.linalg.eigh(M)`.

In [ ]:
# Guess de la matrice de Fock

# Matrice de Fock transformée, F_p

# Diagonalisation de F_p pour les valeurs propres et les vecteurs propres avec NumPy


#### Calcul :
Affichez, c'est à dire, `imprimez`, la matrice des coefficients et confirmez qu'elle a la bonne taille

Maintenant que nous avons les coefficients dans la base transformée, nous devons revenir en arrière et obtenir les coefficients dans la base AO originale.

#### Calcul : 
Utilisez `A` et la formule que vous avez proposée précédemment pour retransformer la matrice des coefficients dans la base AO. Confirmez que la matrice résultante semble raisonnable, c'est-à-dire de taille et d'amplitude similaires.

In [ ]:
# Transform the coefficient matrix back into AO basis


### X. The Density Matrix
Recall, the Fock matrix is 

$$
F = H + 2J - K
$$
where $H$ is the one electron "core" Hamiltonian, $J$ is the Coulomb integral matrix, and $K$ is the exchange integral matrix. The HF energy can be expressed in explicit terms of one and two electron integrals
$$
E_{HF} = \sum_i^{elec}\langle i|h_i|i\rangle + \sum_{i>j}^{elec}[ii|jj]-[ij|ji]
$$
Expanding the orbitals in terms of basis functions, we find
$$
[ii|jj]=\sum_{pqrs}c^*_{pi}c_{qi}c^*_{rj}c_{sj}\int{\rm d}\tau\; \phi_p^*(1)\phi_q(1)\frac{1}{r_{ij}}\phi_r^*(2)\phi_s(2)\qquad{\text{(3)}}
$$


First, look at the coefficients in Eq.3. They come in two pairs of complex-conjugates, $c^*_{pi}c_{qi}$ and $c^*_{ri}c_{si}$. The diagonal terms, when $p=q$ for example, are the probability of some basis function $p$ contributing to the MO $i$. We will sum each term over the occupied orbitals, $i$, to form the "density matrix"
$$
D_{pq}=\sum_i^{occ} c^*_{pi}c_{qi}.
$$

We are going to construct the density matrix from the occupied orbitals.  To get a matrix of just the occupied orbitals, use the coefficient matrix in the original AO basis, and take a slice to include all rows and just the columns that represent the occupied orbitals.

In [ ]:
# Grab occupied orbitals (recall: ndocc is the number of doubly occupied orbitals we found earlier)
C_occ = C[:, :ndocc]
print(F'The shape of C_occ is {C_occ.shape}')

#### Calculate: Build the density matrix, `D`, from the occupied orbitals, `C_occ`, using the function `np.einsum()`. (Recall Section VI.)

In [ ]:
# Build density matrix from occupied orbitals

D = #<your equation here>

print(F'The shape of D is {D.shape}')

### XII. Coulomb and Exchange Integrals and the SCF Energy

The integral on the right of Eq.3 is super important. It has four indicies, $p,q,r,s$, so formally it is a tensor. It accounts for the repulsion between pairs of electrons, so it is called the electron repulsion integral tensor, $I$,
$$
I_{pqrs} = \int{\rm d}\tau\; \phi_p^*(1)\phi_q(1)\frac{1}{r_{ij}}\phi_r^*(2)\phi_s(2).
$$
First, we can build the electron-repulsion integral (ERI) tensor, which stores the electron repulsion between the atomic orbital wavefunctions. Mints does all the work for us!

In [ ]:
# Build electron repulsion integral (ERI) Tensor
I = np.asarray(mints.ao_eri())

Eq.3 can be expressed in terms of $I$ and $D$ as
\begin{align}
[ii|jj] &= \sum_{pqrs}D_{pq}D_{rs}I_{pqrs},\\
&=\sum_{pq}D_{pq}\sum_{rs}D_{rs}I_{pqrs}.
\end{align}
The term $\sum_{rs}D_{rs}I_{pqrs}$ is the effective repulsion felt by one electron due to the other electrons in the system. This term is the Coulomb integral matrix
$$
J_{pq}=\sum_{rs}D_{rs}I_{pqrs}.
$$

#### Calculate: Define J  in terms of the density matrix, `D`, and the electron repulsion integral tensor, `I`, using `np.einsum()`.

In [ ]:
#Define J

Similarly, 
\begin{align}
[ij|ji] &= \sum_{pqrs}D_{ps}D_{rq}I_{pqrs},\\
 &= \sum_{ps}D_{ps}\sum_{rq}D_{rq}I_{pqrs}.
\end{align}
corrects the repulsion due to electrons "avoiding each other" due to their Fermionic (antisymmetric w.r.t. exchange) character. This term is the exchange integral matrix
$$
K_{ps}=\sum_{rq}D_{rq}I_{pqrs}.
$$

#### Calculate: Define K  in terms of the density matrix, `D`, and the electron repulsion integral tensor, `I`, using `einsum()`. 

In [ ]:
#Define K

#### Calculate: Define F in terms of H, J, and K. (Recall Section XI.)

In [ ]:
#Define F as a function of H, J, and K

A more convenient form of the SCF energy is in terms of a sum over the AO basis functions
$$
E =  E_{nuc} + \sum_{pq}(H_{pq} + F_{pq})D_{pq},
$$
where $E_{nuc}$ is the nuclear repulsion, which psi4 can calculate for us as well.

#### Calculate: SCF energy based on H, F, and D using `np.einsum()`. 
*(Hint: The right hand side of the equation is the sum of the product of two terms, each of which has two indices (p and q). The result, E, is a number, so it has no indices. In `einsum()` notation, this case will be represented with the indices for the matrices on the left of the `->`, and no index on the right of the `->`. For example, in the case of just one matrix, the sum of all its elements of a matrix `M` is `sum_of_m = np.einsum('pq->',M )`. In your answer below, be sure to account for any modifications required of an element-wise product of two matrices.)* 

In [ ]:
E_nuc = mol.nuclear_repulsion_energy()
SCF_E = #<your formula here>
print(F'Energy = {SCF_E:.8f}')

#### Question: Based on the result of the calculation in Section VII, is this a reasonable answer? 

**Answer:** 

#### Question: Describe a procedure (i.e. identify the steps/commands) that will updated coefficients and compute a new density matrix based on the new definition of the Fock matrix. (Recall Section X)

**Answer** 

#### Calculate: Using the procedure proposed above, calculate the updated coefficients

In [ ]:
# Update density matrix

You have just completed one cycle of the SCF calculation!


Now we will use the density matrix to build the Fock matrix.  The code block below sets up a skeleton of the Hartree-Fock procedure.  The basic steps are:
1. Calculate the Fock Matrix based on the density matrix previously defined from a one electron hamiltonian
2. Calculate the energy from the Fock matrix.
3. Check and see if the energy has converged by comparing the current energy to the previous energy and seeing if it is within the convergence threshold.
4. If the energy has not converged, transform the Fock matrix, and diagonalize the transformed Fock matrix to get the energy and MO coefficients.  Then transform back to the original AO basis, pull the occupied orbitals, and reconstruct the density matrix. 

In [ ]:
# ==> Nuclear Repulsion Energy <==
E_nuc = mol.nuclear_repulsion_energy()

# ==> SCF Iterations <==
# Pre-iteration energy declarations
SCF_E = 0.0
E_old = 0.0

# ==> Set default program options <==
# We continue recalculating the energy until it converges to the level we specify.  
# The varible `E_conv` is where we set this level of convergence.  We also set a 
# maximum number of iterations so that if our calculation does not converge, it 
# eventually stops and lets us know that it did not converge.  
# Maximum SCF iterations
MAXITER = 40
# Energy convergence criterion
E_conv = 1.0e-6

print('==> Starting SCF Iterations <==\n')

# Begin Iterations
for scf_iter in range(1, MAXITER + 1):
    
    # Build Fock matrix (Section XII)
    
    # <your code here>

    # Compute SCF energy
    
    SCF_E = #<your formula here>
    
    print(F'SCF Iteration {scf_iter}: Energy = {SCF_E:.8f} dE = {SCF_E - E_old:.8f}')
    
    # Check to see if the energy is converged.  If it is break out of the loop.
    # If it is not, set the current energy E_old
    
    if (abs(SCF_E - E_old) < E_conv):
        break
    E_old = SCF_E
    
    # Compute new coefficient & density matrices (Section X & XI) 
    
    # <your code here>  
    
    # MAXITER exceeded?
    if (scf_iter == MAXITER):
        psi4.core.clean()
        raise Exception("Maximum number of SCF iterations exceeded.")

# Post iterations
print('\nSCF converged.')
print(F'Final RHF Energy: {SCF_E:.6f} [Eh]')

Compare your results to Psi4 by computing the energy using `psi4.energy()` in the cell below.  

In [ ]:
# ==> Compare our SCF to Psi4 <==
# Call psi4.energy() to compute the SCF energy
SCF_E_psi = psi4.energy('SCF')
psi4.core.clean()

# Compare our energy value to what Psi4 computes
assert psi4.compare_values(SCF_E_psi, SCF_E, 6, 'My SCF Procedure')

#### Question: Modify the value of E_conv and describe its affect the number of iterations.

### XIII. Using Hartree-Fock to Justify Molecular Structure

Why is CO$_2$ linear? Why is H$_2$O bent? Why is CH$_4$ tetrahedral? Why is FeF$_6$ octahedral? In general
chemistry, we used valence shell electron pair repulsion (VSEPR) theory to justify molecular structures
by invoking a _repulsion_ between both bonding and non-bonding pairs of electrons.  The reality of molecular
structure is more complicated, however.

In this section of the lab, we will use the same Hartree-Fock method that we implemented above to justify the
bent structure of water by computing the electronic energy of H$_2$O at a variety of bond angles.  


In [ ]:
# ==> Scanning a Bond Angle: Flexible Water <==
# Import a library to visualize energy profile
import matplotlib.pyplot as plt
%matplotlib inline

# Define flexible water molecule using Z-matrix
flexible_water = """
O
H 1 0.96
H 1 0.96 2 {}
"""

# Scan over bond angle range between 90 & 180, in 5 degree increments
scan = {}
for angle in range(90, 181, 5):
    # Make molecule
    mol = psi4.geometry(flexible_water.format(angle))
    # Call Psi4
    e = psi4.energy('scf/cc-pvdz', molecule=mol)
    #e = psi4.energy('scf/sto-3g', molecule=mol)

    # Save energy in dictionary
    scan[angle] = e

# Visualize energy profile
x = list(scan.keys())
y = list(scan.values())
plt.plot(x,y,'ro-')
plt.xlabel('H-O-H Bond Angle ($^{\circ}$)')
plt.ylabel('Molecular Energy ($E_h$)')
plt.show()

Using the energy profile we generated above, justify the experimentally measured water bond angle of 104.5$^{\circ}$ in the cell below.

**Answer**